In [6]:
import pandas as pd
from pandas.io import gbq
import numpy as np
import os
import glob
import re

In [7]:
import pydata_google_auth

In [8]:
    connect_bq = [
        'https://www.googleapis.com/auth/cloud-platform',
        'https://www.googleapis.com/auth/drive',
    ]

    credentials = pydata_google_auth.get_user_credentials(
        connect_bq,
        auth_local_webserver=True,
    )

#### Query that retrieves add_ons per user for a month and the add ons that a bundle has

In [9]:
sql_query = """
CREATE TEMP FUNCTION STRING_DEDUP(str STRING) AS (
(SELECT STRING_AGG(item ORDER BY item) FROM (
SELECT DISTINCT item FROM UNNEST(SPLIT(str)) item 
)) 
);
----------------------------------------------------------------------------------------------------------------------
with daily_status as (
Select * , DATE_TRUNC(day, month) as change_month
from `fubotv-prod.data_insights.daily_status_static_update` t1
where final_status_restated like '%paid%'
and day >= '2021-01-01'
AND day <= current_date()-1
),
add_on as (
select account_code, change_month, STRING_DEDUP(add_ons) as addons_sorted,STRING_DEDUP(bundle_add_on_list) as bundle_addons_sorted
FROM daily_status
)
select DISTINCT change_month, account_code, bundle_addons_sorted, addons_sorted
from add_on
"""

In [10]:
df = pd.read_gbq(
    sql_query,
    project_id='fubotv-prod',
    credentials=credentials,
)

#### Checking / Housekeeping

In [11]:
df.head()

,change_month,account_code,bundle_addons_sorted,addons_sorted
0,2021-06-01,5ffb441aa597bb00016ed4b2,"advanced-dvr-1000,fubo-extra-lite,third-screen...","advanced-dvr-1000,deportes,fubo-extra-lite,thi..."
1,2021-06-01,60b27fe0b8af80000110b216,"advanced-dvr-1000,fubo-extra-lite,third-screen...","advanced-dvr-1000,fubo-extra-lite,starz-epix-s..."
2,2022-03-01,5f6fc190137e190001d3de31,"advanced-dvr-250,third-screen","advanced-dvr-250,intl-sports-plus,third-screen"
3,2021-11-01,61551bcc2bd3490001f08797,"advanced-dvr-250,third-screen","advanced-dvr-250,adventure,third-screen"
4,2021-06-01,5cb916477d68180009005d8a,None,"deportes,fubo-extra-lite-preview"


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18296490 entries, 0 to 18296489
Data columns (total 4 columns):
 #   Column                Dtype         
---  ------                -----         
 0   change_month          datetime64[ns]
 1   account_code          object        
 2   bundle_addons_sorted  object        
 3   addons_sorted         object        
dtypes: datetime64[ns](1), object(3)
memory usage: 558.4+ MB


##### Replace null with blanks to ensure edge cases are accounted for

In [13]:
df = df.fillna(" ")

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18296490 entries, 0 to 18296489
Data columns (total 4 columns):
 #   Column                Dtype         
---  ------                -----         
 0   change_month          datetime64[ns]
 1   account_code          object        
 2   bundle_addons_sorted  object        
 3   addons_sorted         object        
dtypes: datetime64[ns](1), object(3)
memory usage: 558.4+ MB


##### Comparing bundle add_ons to the add_ons field and identifying the difference between these two fields and joining each add_on by a comma to a new field called new_addition

In [16]:
df['new_addition'] = [','.join(set(n.split(',')) - set(o.split(','))) 
                                                          for o, n in zip(df.bundle_addons_sorted, df.addons_sorted)]

In [17]:
df

,change_month,account_code,bundle_addons_sorted,addons_sorted,new_addition
0,2021-06-01,5ffb441aa597bb00016ed4b2,"advanced-dvr-1000,fubo-extra-lite,third-screen...","advanced-dvr-1000,deportes,fubo-extra-lite,thi...",deportes
1,2021-06-01,60b27fe0b8af80000110b216,"advanced-dvr-1000,fubo-extra-lite,third-screen...","advanced-dvr-1000,fubo-extra-lite,starz-epix-s...",starz-epix-showtime
2,2022-03-01,5f6fc190137e190001d3de31,"advanced-dvr-250,third-screen","advanced-dvr-250,intl-sports-plus,third-screen",intl-sports-plus
3,2021-11-01,61551bcc2bd3490001f08797,"advanced-dvr-250,third-screen","advanced-dvr-250,adventure,third-screen",adventure
4,2021-06-01,5cb916477d68180009005d8a,,"deportes,fubo-extra-lite-preview","deportes,fubo-extra-lite-preview"
...,...,...,...,...,...
18296485,2022-06-01,609604d1920bc2000111b0d2,"advanced-dvr-1000,fubo-extra-lite,third-screen...","advanced-dvr-1000,fubo-extra-lite,rsn-fee,thir...",rsn-fee
18296486,2022-02-01,61860a1ff1a5510001698c3f,"advanced-dvr-1000,fubo-extra-lite,third-screen...","advanced-dvr-1000,fubo-extra-lite,rsn-fee,thir...",rsn-fee
18296487,2022-02-01,60aacfc53458b60001ae518d,"advanced-dvr-1000,fubo-extra-lite,third-screen...","advanced-dvr-1000,fubo-extra-lite,rsn-fee,thir...",rsn-fee
18296488,2021-11-01,60c5027ebee73400014a1603,"advanced-dvr-1000,fubo-extra-lite,third-screen...","advanced-dvr-1000,fubo-extra-lite,rsn-fee,thir...",rsn-fee


#### Splitting new_addition column into additional columns and exploding them into individual rows

In [18]:
df['new_addition'] = df['new_addition'].str.split(',')
df = df.explode('new_addition').reset_index(drop=True)
cols = list(df.columns)
df = df[cols]

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df = df.drop(['addons_sorted', 'bundle_addons_sorted'], axis=1)

In [ ]:
df

In [ ]:
df = df.drop_duplicates()

In [ ]:
df

#### Aggregating by month and new_addition to count distinct account_code

In [ ]:
df = df.groupby(['change_month', 'new_addition'],  as_index=False ).agg({'account_code': 'nunique'})

In [ ]:
print(df)

In [ ]:
df.to_csv('/Users/nanditanandakumar/Dropbox (fuboTV)/Business Analytics/Team/Nandita/Git/Addons_Project/Count of Users per Add On - By Month.csv', index =False)